# To do:
* compare variable month and years to stata read data to verify reading them correctly
* make github page for this


In [96]:
################### import packages #######################################################
import pandas as pd
import re
import numpy as np
import os
from hrs_functions import *
import copy 
pd.set_option('display.max_columns', None)
## maybe make this more programatic
## each survey has a letter preceeding variable names, increases by 1 each time
var_dict = {
    2008: 'l'
    , 2010: 'm'
    , 2012: 'n'
    , 2014: 'o'
    , 2016: 'p' 
}

#################################### define functions ####################################
def read_yr_of_data(directory, variables_to_look_for, survey_year):
    # list all files to potentially read
    directory_sta = directory + 'h' + str(survey_year)[2:4] + 'sta'
    directory_da = directory + 'h' + str(survey_year)[2:4] + 'da'
    path_list = list(set([directory_sta + '/' +  x.split('.')[0] for x in os.listdir(directory_sta) 
                          if 'dct' in x
                          and 'R' in x[len(x.split('.')[0])-1]
                         ]))
    
    # figure out which surveys to read based on variables to look for
    surveys = [x[0] for x in variables_to_look_for]\
        + [x[0:2] for x in variables_to_look_for if 'lb' in x]
    # find the surveys in the path list
    path_short_list = [x for x in path_list if 'r' in x.split('/')[8].lower() # only get respondent
                       and any(
                           y == x.split('/')[8].lower()[3:len(x.split('/')[8]) - 2] 
                           for y in surveys
                       ) 
                      ]
    
    # these are the id's for all R level surveys in all years
    id_variables = ['hhid', 'pn']
    
    survey_specific_variables = [var_dict[survey_year] + x
                             for x in variables_to_look_for]
    rename_dict = copy.deepcopy(variables_to_look_for)
    vars_list = [x for x in variables_to_look_for]
    
    for x in range(len(survey_specific_variables)):
        rename_dict[survey_specific_variables[x]] = rename_dict.pop(vars_list[x])
    
    # read the files
    for i in range(len(path_short_list)):
        path = path_short_list[i]
        filename = directory_da + '/' + path.split('/')[8] + '.da'
        dct_file = directory_sta + '/' + path.split('/')[8] +  '.dct'
        print(filename)
        try:
            temp_df = read_stata_files(filename, dct_file)
            temp_df = temp_df[[x for x in temp_df.columns if x.lower() in (id_variables + survey_specific_variables)]]
            temp_df = temp_df.rename(columns = rename_dict)
            ## maybe rename them here? some sort of naming dictionary
            
            if i == 0:
                year_frame = temp_df
            else:
                year_frame = year_frame.merge(temp_df, on = ["hhid", "pn"], how = "outer")
        except:
            print('missing files: ' + path)
            pass
        
    # make the wave variable, 1992 was wave 1
    wave = (survey_year - 1990) / 2
    year_frame['wave'] = wave
    return year_frame

In [2]:
cwd = os.getcwd()
variables_to_look_for = {'a500':'rmonth_survey'
                         , 'a501':'ryear_survey'
                         , 'j073m1':'rjobwhym1'
                         , 'lb032_2':'rrisk_financial'
                         , 'n106':'rhosp_cost_oop'}
survey_year = 2016
directory = cwd + '/data/'

for yr in [2014, 2016]:
    frame = read_yr_of_data(directory, variables_to_look_for, yr)
    print(frame)

/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14J_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14LB_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14N_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14L_R.da
/Users/jcoven/Desktop/HRS_Assembly/python/data/h14da/H14A_R.da
         hhid   pn  rjobwhym1  rrisk_financial  rhosp_cost_oop  rmonth_survey  \
0          30  200        NaN             20.0             NaN             52   
1      100010  100        NaN              NaN             NaN             42   
2      100030  300        NaN             40.0             NaN             32   
3      100040  400        NaN             60.0             NaN             42   
4      100130  401        NaN             40.0             NaN             42   
5      100130  102        NaN              NaN             NaN             82   
6      100380  100        NaN             80.0             NaN             42   
7      100380  400        NaN       

In [6]:
frame.groupby(['rmonth_survey', 'ryear_survey']).count()

filename = '/Users/jcoven/Desktop/HRS_Assembly/python/data/h16da/H16A_R.da'
dct_file = '/Users/jcoven/Desktop/HRS_Assembly/python/data/h16sta/H16A_R.dct'
df = read_stata_files(filename, dct_file)

In [33]:
df.loc[df.hhid == 10001]

,hhid,pn,psubhh,osubhh,ppn_sp,pcsr,pfamr,pfinr,pa500,pa501,...,pa065,pa066,pa068m,pa099,pa100,pa101,pa106,pa113,pvdate,pversion


In [25]:
test = pd.read_stata('/Users/jcoven/Desktop/H16A_R.dta')

In [74]:
test

,HHID,PN,PSUBHH,OSUBHH,PPN_SP,PCSR,PFAMR,PFINR,PA500,PA501,PA061,PA062,PA063,PA064,PA002,PA009,PA215,PA010,PA103,PA011,PA012,PA019,PA216,PA218,PA219,PA028,PA065,PA066,PA068M,PA099,PA100,PA101,PA106,PA113,PVDATE,PVERSION
0,010001,010,0,0,,1,1,1,5,2016,2015,2014,29,4.0,1,1,NaN,NaN,NaN,NaN,1,76,NaN,NaN,NaN,5,NaN,NaN,NaN,0,0,0,0,0,3,2
1,010003,030,0,0,,1,1,1,3,2016,2015,2014,27,3.0,1,1,NaN,NaN,NaN,NaN,1,60,NaN,NaN,NaN,5,NaN,NaN,NaN,0,0,0,0,0,2,2
2,010004,040,0,0,,1,1,1,5,2016,2015,2014,29,4.0,1,1,NaN,NaN,NaN,NaN,1,70,NaN,NaN,NaN,5,NaN,NaN,NaN,0,2,2,2,4,3,2
3,010013,040,1,1,,1,1,1,3,2016,2015,2014,27,4.0,1,1,NaN,NaN,NaN,NaN,1,68,NaN,NaN,NaN,5,NaN,NaN,NaN,0,1,1,1,2,2,2
4,010038,010,0,0,040,5,5,1,4,2016,2015,2014,28,4.0,1,1,NaN,NaN,NaN,NaN,1,79,NaN,NaN,NaN,5,NaN,NaN,NaN,0,2,2,2,4,2,2
5,010038,040,0,0,010,1,1,5,4,2016,2015,2014,28,4.0,1,1,NaN,NaN,NaN,NaN,1,73,NaN,NaN,NaN,5,NaN,NaN,NaN,0,2,2,2,4,2,2
6,010050,010,0,0,,1,1,1,12,2016,2015,2014,36,4.0,1,1,NaN,NaN,NaN,NaN,1,75,NaN,NaN,NaN,5,NaN,NaN,NaN,0,0,0,0,0,8,2
7,010059,020,0,0,030,1,1,5,6,2016,2015,2014,30,3.0,1,1,NaN,NaN,NaN,NaN,1,81,NaN,NaN,NaN,5,NaN,NaN,NaN,0,3,3,3,6,4,2
8,010059,030,0,0,020,5,5,1,12,2016,2015,2014,36,4.0,1,1,NaN,NaN,NaN,NaN,1,88,NaN,NaN,NaN,5,NaN,NaN,NaN,0,3,3,3,6,9,2
9,010063,010,0,0,,1,1,1,4,2017,2016,2015,28,NaN,1,1,NaN,NaN,NaN,NaN,2,79,NaN,NaN,NaN,1,10.0,2008.0,2.0,0,1,1,1,2,10,2


In [75]:
df

,hhid,pn,psubhh,osubhh,ppn_sp,pcsr,pfamr,pfinr,pa500,pa501,pa061,pa062,pa063,pa064,pa002,pa009,pa215,pa010,pa103,pa011,pa012,pa019,pa216,pa218,pa219,pa028,pa065,pa066,pa068m,pa099,pa100,pa101,pa106,pa113,pvdate,pversion
0,010001,010,0,0,NaN,1,1,1,5,2016,2015,2014,29,4,1,1,NaN,NaN,NaN,NaN,1,76,5,NaN,NaN,NaN,NaN,0,0,0,0,0,3,1,NaN,NaN
1,010003,030,0,0,NaN,1,1,1,3,2016,2015,2014,27,3,1,1,NaN,NaN,NaN,NaN,1,60,5,NaN,NaN,NaN,NaN,0,0,0,0,0,2,1,NaN,NaN
2,010004,040,0,0,NaN,1,1,1,5,2016,2015,2014,29,4,1,1,NaN,NaN,NaN,NaN,1,70,5,NaN,NaN,NaN,NaN,0,2,2,2,4,3,1,NaN,NaN
3,010013,040,1,1,NaN,1,1,1,3,2016,2015,2014,27,4,1,1,NaN,NaN,NaN,NaN,1,68,5,NaN,NaN,NaN,NaN,0,1,1,1,2,2,1,NaN,NaN
4,010038,010,0,0,040,5,5,1,4,2016,2015,2014,28,4,1,1,NaN,NaN,NaN,NaN,1,79,5,NaN,NaN,NaN,NaN,0,2,2,2,4,2,1,NaN,NaN
5,010038,040,0,0,010,1,1,5,4,2016,2015,2014,28,4,1,1,NaN,NaN,NaN,NaN,1,73,5,NaN,NaN,NaN,NaN,0,2,2,2,4,2,1,NaN,NaN
6,010050,010,0,0,NaN,1,1,1,12,2016,2015,2014,36,4,1,1,NaN,NaN,NaN,NaN,1,75,5,NaN,NaN,NaN,NaN,0,0,0,0,0,8,1,NaN,NaN
7,010059,020,0,0,030,1,1,5,6,2016,2015,2014,30,3,1,1,NaN,NaN,NaN,NaN,1,81,5,NaN,NaN,NaN,NaN,0,3,3,3,6,4,1,NaN,NaN
8,010059,030,0,0,020,5,5,1,12,2016,2015,2014,36,4,1,1,NaN,NaN,NaN,NaN,1,88,5,NaN,NaN,NaN,NaN,0,3,3,3,6,9,1,NaN,NaN
9,010063,010,0,0,NaN,1,1,1,4,2017,2016,2015,28,NaN,1,1,NaN,NaN,NaN,NaN,2,79,1,10,2,0,8,200,1,1,1,2,10,1,NaN,NaN


In [95]:
variables.loc[~variables.name.str.contains('a216')
              & ~variables.name.str.contains('a218')
             ]

,start,type,name,fstring,desc,end
0,0,<class 'str'>,hhid,%6s,HOUSEHOLD IDENTIFICATION NUMBER,6.0
1,6,<class 'str'>,pn,%3s,RESPONDENT PERSON IDENTIFICATION NUMBER,9.0
2,9,<class 'str'>,psubhh,%1s,2016 SUB HOUSEHOLD IDENTIFICATION NUMBER,10.0
3,10,<class 'str'>,osubhh,%1s,2014 SUB HOUSEHOLD IDENTIFICATION NUMBER,11.0
4,11,<class 'str'>,ppn_sp,%3s,2016 SPOUSE/PARTNER PERSON NUMBER,14.0
5,14,<class 'int'>,pcsr,%1.0f,2016 WHETHER COVERSHEET RESPONDENT,15.0
6,15,<class 'int'>,pfamr,%1.0f,2016 WHETHER FAMILY RESPONDENT,16.0
7,16,<class 'int'>,pfinr,%1.0f,2016 WHETHER FINANCIAL RESPONDENT,17.0
8,17,<class 'int'>,pa500,%2.0f,DATE OF INTERVIEW - MONTH,19.0
9,19,<class 'int'>,pa501,%4.0f,DATE OF INTERVIEW - YEAR,23.0


In [97]:
filename = '/Users/jcoven/Desktop/HRS_Assembly/python/data/h16da/H16A_R.da'
dct_file = '/Users/jcoven/Desktop/HRS_Assembly/python/data/h16sta/H16A_R.dct'


# code adapted from Allen B. Downey, Think Stats: Exploratory Data Analysis
type_map = dict(byte=int, int=int, long=int, float=float, 
                    double=float, numeric=float)
index_base = 0
var_info = []
with open(dct_file) as f:
    for line in f:
        match = re.search( r'_column\(([^)]*)\)', line)
        if not match:
            continue
        start = int(match.group(1))
        t = line.split()
        vtype, name, fstring = t[1:4]
        name = name.lower()
        if vtype.startswith('str'):
            vtype = str
        else:
            vtype = type_map[vtype]
        long_desc = ' '.join(t[4:]).strip('"')
        var_info.append((start, vtype, name, fstring, long_desc))

columns = ['start', 'type', 'name', 'fstring', 'desc']
variables = pd.DataFrame(var_info, columns=columns)

# fill in the end column by shifting the start column
variables['end'] = variables.start.shift(-1)
variables['start'] = variables['start'] - 1 # new addition, because 0 indexed unlike stata
variables['end'] = variables['end'] - 1 # new addition, because 0 indexed unlike stata
variables.loc[len(variables)-1, 'end'] = 0

colspecs = variables[['start', 'end']] - index_base
colspecs = colspecs.astype(np.int).values.tolist()

names = variables['name']

df = pd.read_fwf(filename, colspecs=colspecs, names = names
                 , converters = test_dict
                )
df

,hhid,pn,psubhh,osubhh,ppn_sp,pcsr,pfamr,pfinr,pa500,pa501,pa061,pa062,pa063,pa064,pa002,pa009,pa215,pa010,pa103,pa011,pa012,pa019,pa216,pa218,pa219,pa028,pa065,pa066,pa068m,pa099,pa100,pa101,pa106,pa113,pvdate,pversion
0,010001,010,0,0,NaN,1,1,1,5,2016,2015,2014,29,4,1,1,NaN,NaN,NaN,NaN,1,76,5,NaN,NaN,NaN,NaN,0,0,0,0,0,3,1,NaN,NaN
1,010003,030,0,0,NaN,1,1,1,3,2016,2015,2014,27,3,1,1,NaN,NaN,NaN,NaN,1,60,5,NaN,NaN,NaN,NaN,0,0,0,0,0,2,1,NaN,NaN
2,010004,040,0,0,NaN,1,1,1,5,2016,2015,2014,29,4,1,1,NaN,NaN,NaN,NaN,1,70,5,NaN,NaN,NaN,NaN,0,2,2,2,4,3,1,NaN,NaN
3,010013,040,1,1,NaN,1,1,1,3,2016,2015,2014,27,4,1,1,NaN,NaN,NaN,NaN,1,68,5,NaN,NaN,NaN,NaN,0,1,1,1,2,2,1,NaN,NaN
4,010038,010,0,0,040,5,5,1,4,2016,2015,2014,28,4,1,1,NaN,NaN,NaN,NaN,1,79,5,NaN,NaN,NaN,NaN,0,2,2,2,4,2,1,NaN,NaN
5,010038,040,0,0,010,1,1,5,4,2016,2015,2014,28,4,1,1,NaN,NaN,NaN,NaN,1,73,5,NaN,NaN,NaN,NaN,0,2,2,2,4,2,1,NaN,NaN
6,010050,010,0,0,NaN,1,1,1,12,2016,2015,2014,36,4,1,1,NaN,NaN,NaN,NaN,1,75,5,NaN,NaN,NaN,NaN,0,0,0,0,0,8,1,NaN,NaN
7,010059,020,0,0,030,1,1,5,6,2016,2015,2014,30,3,1,1,NaN,NaN,NaN,NaN,1,81,5,NaN,NaN,NaN,NaN,0,3,3,3,6,4,1,NaN,NaN
8,010059,030,0,0,020,5,5,1,12,2016,2015,2014,36,4,1,1,NaN,NaN,NaN,NaN,1,88,5,NaN,NaN,NaN,NaN,0,3,3,3,6,9,1,NaN,NaN
9,010063,010,0,0,NaN,1,1,1,4,2017,2016,2015,28,NaN,1,1,NaN,NaN,NaN,NaN,2,79,1,10,2,0,8,200,1,1,1,2,10,1,NaN,NaN


In [99]:

colspecs

[[0, 6],
 [6, 9],
 [9, 10],
 [10, 11],
 [11, 14],
 [14, 15],
 [15, 16],
 [16, 17],
 [17, 19],
 [19, 23],
 [23, 27],
 [27, 31],
 [31, 33],
 [33, 35],
 [35, 36],
 [36, 37],
 [37, 38],
 [38, 39],
 [39, 41],
 [41, 42],
 [42, 43],
 [43, 46],
 [46, 48],
 [48, 50],
 [50, 51],
 [51, 52],
 [52, 54],
 [54, 58],
 [58, 60],
 [60, 62],
 [62, 64],
 [64, 66],
 [66, 68],
 [68, 70],
 [70, 72],
 [72, 0]]

In [77]:
test_dict = {}
for i in variables['name']:
    test_dict[i] = variables.loc[variables.name == i, 'type'].iloc[0]
test_dict

{'hhid': str,
 'pn': str,
 'psubhh': str,
 'osubhh': str,
 'ppn_sp': str,
 'pcsr': int,
 'pfamr': int,
 'pfinr': int,
 'pa500': int,
 'pa501': int,
 'pa061': int,
 'pa062': int,
 'pa063': int,
 'pa064': int,
 'pa002': int,
 'pa009': int,
 'pa215': int,
 'pa010': int,
 'pa103': int,
 'pa011': int,
 'pa012': int,
 'pa019': int,
 'pa216': int,
 'pa218': int,
 'pa219': int,
 'pa028': int,
 'pa065': int,
 'pa066': int,
 'pa068m': int,
 'pa099': int,
 'pa100': int,
 'pa101': int,
 'pa106': int,
 'pa113': int,
 'pvdate': int,
 'pversion': int}

In [101]:
type(names)

pandas.core.series.Series

In [104]:
test_dict['hhid']

TypeError: unsupported operand type(s) for +: 'type' and 'str'